<img src="https://raw.githubusercontent.com/autonomousvision/navsim/main/assets/navsim_transparent.png" alt="drawing" width="800"/>

# NAVSIM 可视化教程

本教程将介绍一些基本的绘图方法来可视化 NAVSIM 中的驾驶场景。所有图表都使用 `matplotlib` 创建，易于根据你的应用进行自定义。

## 目录
1. [配置](#config)
2. [鸟瞰图（BEV）](#bev)
3. [相机视图](#camera)
4. [创建自定义图表](#custom)
5. [创建 GIF 动图](#gifs)

## 配置 <a name="config"></a>

NAVSIM 提供两种类型的可视化： 
- 鸟瞰图（Birds-Eye-View，BEV）或 
- 相机图像。 

LiDAR 传感器可以在 BEV 或相机图像中可视化。所有图表在 [`navsim/visualization/config.py`](https://github.com/autonomousvision/navsim/blob/main/navsim/navsim/visualization/config.py) 中有全局配置。在这个 Python 文件中，你可以配置所有颜色或尺寸。LiDAR 点云可以使用任何颜色映射进行着色，显示到自车的距离或每个点的高度。在本教程中，我们首先从 mini 数据集实例化 `SceneFilter` 和 `SceneLoader`。

In [ ]:
import os
from pathlib import Path

import hydra
from hydra.utils import instantiate
import numpy as np
import matplotlib.pyplot as plt

from navsim.common.dataloader import SceneLoader
from navsim.common.dataclasses import SceneFilter, SensorConfig

SPLIT = "mini"  # ["mini", "test", "trainval"]
FILTER = "all_scenes"

hydra.initialize(config_path="./navsim/planning/script/config/common/train_test_split/scene_filter")
cfg = hydra.compose(config_name=FILTER)
scene_filter: SceneFilter = instantiate(cfg)
openscene_data_root = Path(os.getenv("OPENSCENE_DATA_ROOT"))

scene_loader = SceneLoader(
    openscene_data_root / f"navsim_logs/{SPLIT}", # data_path
    openscene_data_root / f"sensor_blobs/{SPLIT}", # original_sensor_path
    scene_filter,
    openscene_data_root / "warmup_two_stage/sensor_blobs", # synthetic_sensor_path
    openscene_data_root / "warmup_two_stage/synthetic_scene_pickles", # synthetic_scenes_path
    sensor_config=SensorConfig.build_all_sensors(),
)

## 鸟瞰图（BEV） <a name="bev"></a>

NAVSIM 中的鸟瞰图（BEV）可视化对于查看地图、边界框标注或 LiDAR 点云非常有用。在标准设置下，BEV 图显示以自车后轴为中心的 64m × 64m 区域（为简化起见不包括 LiDAR）。首先，我们随机选择一个 token 并加载一个场景进行可视化。

In [ ]:
token = np.random.choice(scene_loader.tokens)
scene = scene_loader.get_scene_from_token(token)

函数 `plot_bev_frame` 接受一个 `Scene` 对象和要可视化的步骤索引（历史或未来）。

In [ ]:
from navsim.visualization.plots import plot_bev_frame

frame_idx = scene.scene_metadata.num_history_frames - 1 # 当前帧
fig, ax = plot_bev_frame(scene, frame_idx)
plt.show()

函数 `plot_bev_with_agent` 在当前帧将 <span style="color:#DE7061">智能体</span> 的轨迹与 <span style="color:#59a14f">人类驾驶员</span> 的轨迹进行对比可视化。本教程展示了一个简单的 `ConstantVelocityAgent`（恒定速度智能体）示例：

In [ ]:
from navsim.visualization.plots import plot_bev_with_agent
from navsim.agents.constant_velocity_agent import ConstantVelocityAgent

agent = ConstantVelocityAgent()
fig, ax = plot_bev_with_agent(scene, agent)
plt.show()

## 相机视图 <a name="camera"></a>

NAVSIM 中的智能体可以访问环绕车辆的八个相机。函数 `plot_cameras_frame` 以 3 × 3 网格显示相机，各个方向的相机图像围绕中心的 BEV 图排列。

In [ ]:
from navsim.visualization.plots import plot_cameras_frame

fig, ax = plot_cameras_frame(scene, frame_idx)
plt.show()

使用 `plot_cameras_frame_with_annotations`，你可以在相机图像中可视化边界框标注。

In [ ]:
from navsim.visualization.plots import plot_cameras_frame_with_annotations

fig, ax = plot_cameras_frame_with_annotations(scene, frame_idx)
plt.show()

使用 `plot_cameras_frame_with_lidar`，你可以在相机图像中可视化 LiDAR 点云。

In [ ]:
from navsim.visualization.plots import plot_cameras_frame_with_lidar

fig, ax = plot_cameras_frame_with_lidar(scene, frame_idx)
plt.show()

## 创建自定义图表 <a name="custom"></a>

NAVSIM 中的图表使用 `matplotlib`，可以向 `plt.Axes` 对象添加元素或返回完整的 `plt.Figure`。[`navsim/visualization/`](https://github.com/autonomousvision/navsim/blob/main/navsim/navsim/visualization) 中的函数可以重复使用来创建自定义图表。在这个例子中，我们创建了一个包含边界框标注和 LiDAR 点云的图表。

In [ ]:
from navsim.visualization.plots import configure_bev_ax
from navsim.visualization.bev import add_annotations_to_bev_ax, add_lidar_to_bev_ax


fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.set_title("自定义图表")

add_annotations_to_bev_ax(ax, scene.frames[frame_idx].annotations)
add_lidar_to_bev_ax(ax, scene.frames[frame_idx].lidar)

# 配置为 BEV 视图
configure_bev_ax(ax)

plt.show()

## 创建 GIF 动图 <a name="gifs"></a>

你可以将逐帧图表转换为简短的 GIF 动图。将任何接受 `Scene` 和 `frame_idx` 作为输入的函数传递给 `frame_plot_to_gif`（例如 `plot_cameras_frame_with_annotations`）。

In [ ]:
from navsim.visualization.plots import frame_plot_to_gif

frame_indices = [idx for idx in range(len(scene.frames))]  # 场景中的所有帧
file_name = f"./{token}.gif"
images = frame_plot_to_gif(file_name, plot_cameras_frame_with_annotations, scene, frame_indices)